### MSCI 531 Project

This linear program implementation refers to the paper "__optimal ordering policies for a perishable and substututable product: A Markov decision model__" by Mahmut Parlar.

In [ ]:
import numpy as np
from scipy.stats import binom
import random as rd
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Constants
s_0 = 10 # sale price of opo items
s_1 = 16 # sale price of new items s_1 > s_0
pi_0 = 0.5 #
pi_1 = 0.8 #
c = 8 # cost to order a unit
a_01 = 0.4 # probability a customer will accept a new items when opo is out of stock
a_10 = 0.7 # probability a customer will accept opo items when new items are out of stock
M = 7 # Number of periods to go till
K = 7 # maximum number of new items that can be ordered

# demands of opo items
d_0 = [binom.pmf(i,K,pi_0) for i in range(0,K+1)]

# demands of new items
d_1 = [binom.pmf(i,K,pi_1) for i in range(0,K+1)]

In [ ]:
# Rewards Z(i,k)

def profit(q_t,Q_t:int):
    # region 1
    r1 = s_1*sum(i*d_1[i] for i in range(0,Q_t+1)) + s_1*sum(Q_t*d_1[i] for i in range(Q_t+1,K+1)) - c*Q_t
    
    # region 2
    r2 = sum(sum(s_0*min(a_10*(y-Q_t),q_t-x)*d_0[x]*d_1[y] for x in range(0,q_t+1)) for y in range(Q_t+1,K+1))
    
    # region 3
    r3 = sum(sum(s_1*min(a_01*(x-q_t),Q_t-y)*d_0[x]*d_1[y] for x in range(q_t+1,K+1)) for y in range(0,Q_t+1))
    
    #region 4
    r4 = s_0*M*pi_0 - s_0*sum(x*d_0[x] for x in range(q_t+1,K+1)) + s_0*q_t*(1-sum(d_0[i] for i in range(0,q_t+1)))
    
    return r1+r2+r3+r4

In [ ]:
# Probabilities Pij(k)

# P(X=x, Y=y)
pxy = {}
for x in range(0,K+1):
    for y in range(0,K+1):
        pxy[x,y] = binom.pmf(x,K,pi_0) * binom.pmf(y,K,pi_1)

def prob(i,j,k):
    # if the cell in pxy array equals to value v, then add it to the total probability
    final_Prob = 0
    for x in range(0,K+1):
        for y in range(0,K+1):
            v = k-y-max(0,a_01*(x-i))
            if(v == j):
                final_Prob = final_Prob + pxy[x,y]
            else:
                continue;
                
    return final_Prob

In [ ]:
model = gp.Model('531 Project')

In [ ]:
Zik = {}
for i in range(0,K+1):
    for k in range(0,K+1):
            Zik[i,k] = profit(i,k)

Pijk = {}
period = 0
for i in range(0,K+1):
    for j in range(0,K+1):
        for k in range(0,K+1):
            Pijk[i,j,k] = prob(i,j,k)
    period = period + 1

# Decision variable
Yik = {}
for i in range(0,K+1):
    for k in range(0,K+1):
        Yik[i,k] = model.addVar(vtype=GRB.CONTINUOUS, lb=0.0, ub=1.0, name='Y'+str(i)+str(k))
                

In [ ]:
# Constraints

# probabilities must equal to 1
model.addConstr(sum(sum(Yik[i,k] for k in range(0,K+1)) for i in range(0,M+1)) == 1)

# 
for j in range(0,M):
     model.addConstr((sum(Yik[j,k] for k in range(0,K+1))-sum(sum(Yik[i,k]*Pijk[i,j,k] for k in range(0,K+1)) for i in range(0,M+1)) == 0))

In [ ]:
obj = sum(sum(Zik[i,k]*Yik[i,k] for k in range(0,K)) for i in range(0,K))
model.setObjective(obj, GRB.MAXIMIZE)

# Run optimization engine
model.optimize()

In [ ]:
decisions = []
        
sum=0 
for v in model.getVars():
    if(v.x > 0):
        decisions.append([v.varName,v.x])
    sum = sum + v.x
print('Obj: %g' % model.objVal)

for i in range(0,len(decisions)):
    print(f'{decisions[i][0]} -> {decisions[i][1]}')